# CRF及numpy矩阵运算

## 参考

[简明条件随机场CRF介绍（附带纯Keras实现） - 科学空间|Scientific Spaces](https://kexue.fm/archives/5542)

[bojone/crf: keras implementation of conditional random field](https://github.com/bojone/crf)

In [62]:
import numpy as np

In [63]:
num_labels = 4 # 没有无效标记
trans = np.random.normal(size=(num_labels, num_labels)); trans 

array([[-1.50413628, -0.68863308, -0.82031784, -1.73656058],
       [ 1.55822081,  1.08684108,  0.91119449,  0.20332897],
       [ 2.13036346,  1.07923045,  0.08253849,  1.46765686],
       [ 1.90493853,  0.48238535, -0.52394967, -0.31904003]])

### utils

In [64]:
def index2onehot(index, num=None):
    index = np.array(index)
    assert index.ndim == 1
    num = max(index) + 1 if num is None else num 
    mn = min(index)
    mx = max(index)
    test = lambda x: 0 < x and x < num
    err_str = lambda x: "wrong value %s \
                    in index with num %s"%(x, num)
    assert test(mn), err_str(mn)
    assert test(mx), err_str(mx)

    return np.eye(num)[index]
index2onehot([1,2])
# index2onehot(1)

array([[0., 1., 0.],
       [0., 0., 1.]])

In [65]:
def onehot2index(onehot):
    onehot = np.array(onehot)
    assert np.alltrue( np.sum(abs(onehot), -1) == 1 ), "should be onehot"
    return np.argmax(onehot, -1)
assert onehot2index([0,0,1]) == 2
assert np.alltrue(
    onehot2index(
    [
        [1, 0, 0],
        [0, 1, 0],
    ]
) == [0, 1])

# test CRF.path_score

## init

In [3]:
### y_pred -> inputs@(samples_len, max_seq_len, label_len )
# y_true -> labels@(samples_len, max_seq_len, label_len )
inputs = np.array(
[
    # two samples
        # seq_len 2, max_len 3
        [
            # 4 tags
            [0.1, 0.4, 0.5, 0],
            [0.1, 0.4, 0.5, 0],
            [0,0,0, 1],
        ],
        # seq_len 2, max_len 3
        [
            # 4 tags
            [0.2, 0.4, 0.5, 0],
            [0.2, 0.4, 0.5, 0],
            [0, 0, 0,  1],
        ],
]
)

In [110]:
LABELS=[
    [1,2,3],
    [3,3,3],
]

In [111]:
from functools import partial
f = partial(index2onehot, num=4)
labels = map(f, LABELS)
labels  = np.array(labels)

In [5]:
inputs.shape, labels.shape

((2, 3, 4), (2, 3, 4))

## point_score

point_score = $\sum_{k=1}^{n}{h(y_k;\boldsymbol{x})}$

In [6]:
mask_inputs_by_labels = inputs*labels;
# shape sample, max_seq_len, tags_num
mask_inputs_by_labels, mask_inputs_by_labels.shape 
# takes those matters in y_pred

(array([[[0.1, 0. , 0. , 0. ],
         [0. , 0.4, 0. , 0. ],
         [0. , 0. , 0. , 1. ]],
 
        [[0. , 0.4, 0. , 0. ],
         [0. , 0.4, 0. , 0. ],
         [0. , 0. , 0. , 1. ]]]), (2, 3, 4))

In [7]:
sum_tags_scroe = np.sum(mask_inputs_by_labels,2);
sum_tags_scroe, sum_tags_scroe.shape 
# sum tags score, shape (sample, max_seq_len) 

(array([[0.1, 0.4, 1. ],
        [0.4, 0.4, 1. ]]), (2, 3))

In [8]:
sum_seq_score = np.sum(sum_tags_scroe, 1,
                       keepdims=True # keep the dim as 1
                      ); sum_seq_score, sum_seq_score.shape
# shape, (sample, 1)

(array([[1.5],
        [1.8]]), (2, 1))

In [9]:
point_score = sum_seq_score

## trans_score 

$trans\_score = \sum_{k=1}^{n-1}{g(y_k,y_{k+1})}$

In [10]:
labels.shape, labels
# labels1, labels2，在max_seq_len轴上错开一位

((2, 3, 4), array([[[1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1]],
 
        [[0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1]]]))

expand_dims 's axis is different for labels1 and labels2 so that `new_labels.shape == (sample_len, max_seq_len ,num_labels - 1, num_labels -1)`

In [22]:
# vector v @ transpose(w) multiply trick
tmp1 = np.arange(1,4)
tmp2 = np.arange(-4, -1)
print(tmp1, tmp2)
print()

# 矩阵乘法，行列一个dim都不能少
tmp1 = tmp1.reshape(3,1)
tmp2 = tmp2.reshape(3,1)

ans1 = tmp2 @ np.transpose(tmp1) # 1式
# 布局不一样，保证先行再列
print(ans1, ans1[2,1])


print()

tmp1 = np.arange(1,4)
tmp2 = np.arange(-4, -1)
# 布局和np有点不一样，最里面是列
tmp1 = np.expand_dims(tmp1, -2) # 留出空间，允许广播，当列
tmp2 = np.expand_dims(tmp2, -1) # 扩展最后一维的，相当于转置的，不同一个元素按顺序给不同列作系数，构成方阵
# 扩展倒数第二维的做模板，相当于1式的np.transpose(tmp1)，
# 扩展倒数第一维的做系数，相当于1式的np.transpose(tmp2)
# [ 系数1*[模板], 系数2*[模板], ... ]
# 这样做的好处是整体逻辑还是按位乘法，方便带batch_size和seq_len的时候微操
ans2 = tmp1 * tmp2
print(ans2, ans2[2,1])

assert np.allclose(ans1, ans2)

[1 2 3] [-4 -3 -2]

[[ -4  -8 -12]
 [ -3  -6  -9]
 [ -2  -4  -6]] -4

[[ -4  -8 -12]
 [ -3  -6  -9]
 [ -2  -4  -6]] -4


In [100]:
# onehot下 取列index
t1 = labels[:, :-1]
labels1 = np.expand_dims(t1, 3); 
# labels1.shape, labels1

In [101]:
# onehot下 取行index
t2 = labels[:, 1:]
labels2 = np.expand_dims(t2, 2); 
# labels2.shape, labels2

In [127]:
new_labels = labels1 * labels2

$[y_i, y_{i+1}] = 1$, else = 0, 0<=i<num

In [126]:
# print(LABELS[0])
for sen, tok_idx in zip(new_labels, LABELS):
    sen= np.copy(sen)
    
    for i, tok_id in enumerate(tok_idx[:-1]):
        
        nxt_tok_id = tok_idx[i+1]
        sen[i][tok_id, nxt_tok_id] -= 1
        
    assert np.allclose(sen, 0), sen

In [143]:
t_trans = trans[None, None, :] # 加入batch_size, seq_len
assert t_trans.ndim == new_labels.ndim # 对齐
tmp = t_trans * new_labels  # 最后两维是只有一个非零元的矩阵
tmp1 = np.sum(tmp, (-1, -2) ) # 把这个元素拿出来
tmp2 = np.sum(tmp1, -1, keepdims=True)
tmp2 # 每一个batch剩下一个分数
trans_score = tmp2

### sum up

In [146]:
trans_score , point_score

(array([[ 2.37885135],
        [-0.63808006]]), array([[1.5],
        [1.8]]))

In [145]:
path_score = trans_score + point_score
path_score 

array([[3.87885135],
       [1.16191994]])

# test CRF.loss